# Predict People Satisfaction Across the Globe

Problem Statement:

We would like to build a model that predicts satisfaction score for people of different countries given their country GDP.

# Download Dataset

Download the Better Life Index data (latest edition, currently it is 2017) from the [OECD’s website](http://homl.info/4) as well as stats about GDP per capita from the [IMF’s website](http://homl.info/5). Then you join the tables and sort by GDP per capita. Table 1-1 shows an excerpt of what you get

Alternatively, look into your class google drive for week 5 and download the data from the folder and copy them in your Google Drive.

# Import Dataset to Google Colab

1. Download CSV and XLS files to your computer
2. Upload them to your Google Drive
3. Open the CSV files using Google Sheets so Google will create the dataset in format of Google Sheets
4. You can remove CSV and XLS files from your drive
5. Use the step by step guide from [here](https://colab.research.google.com/notebooks/io.ipynb#scrollTo=vz-jH8T_Uk2c) and scroll down to **" Google Sheets" ** cell to import data into dataframe

NOTE: After creating Google Sheet into your Drive, make sure you are converting Column 2015 to 0.00 format before importing it into Colab  otherwise Google will import it as a string and you will have hard time to clean the data




In [2]:
# Run below line of code for the first time to install gspread. Once installed comment it for future use
#!pip install --upgrade -q gspread
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

# Use gc to open Google Sheet Datasets

In [3]:
#Open given sheet
worksheet = gc.open('BLI_30012019054825599').sheet1

# Read contents of CSV file
bli_rows = worksheet.get_all_values()

# Convert to a DataFrame and render.
import pandas as pd
bli  = pd.DataFrame.from_records(bli_rows, columns = bli_rows[0])

# Remove rows where inequality has values other than TOT
bli = bli[bli["INEQUALITY"]=="TOT"]

# Reformat data based on "indicator column"
bli = bli.pivot(index="Country", columns="Indicator", values="Value")

bli.head()

Indicator,Air pollution,Dwellings without basic facilities,Educational attainment,Employees working very long hours,Employment rate,Feeling safe walking alone at night,Homicide rate,Household net adjusted disposable income,Household net financial wealth,Housing expenditure,Labour market insecurity,Life expectancy,Life satisfaction,Long-term unemployment rate,Personal earnings,Quality of support network,Rooms per person,Self-reported health,Stakeholder engagement for developing regulations,Student skills,Time devoted to leisure and personal care,Voter turnout,Water quality,Years in education
Country,,,,,,,,,,,,,,,,,,,,,,,,
Australia,5,1.1,80,13.2,72,63.6,1,33417,57462,20,4.3,82.5,7.3,1.36,52063,94,2.3,85,2.7,502,14.35,91,92,21.2
Austria,16,1,85,6.78,72,80.7,0.4,32544,59574,21,2.7,81.3,7,1.94,48295,92,1.6,70,1.3,492,14.55,75,93,17.1
Belgium,15,2.3,75,4.31,62,70.7,1,29968,104084,21,4.8,81.1,6.9,3.98,49587,92,2.2,75,2.2,503,15.77,89,84,18.2
Brazil,10,6.7,49,7.15,64,37.3,27.6,12227,7102,20,4.9,74.7,6.6,3.37,14024,90,0.8,70,2.2,395,14.45,79,72,15.9
Canada,7,0.2,91,3.73,73,80.9,1.4,29850,85758,22,3.9,81.5,7.3,0.81,48403,93,2.5,88,3,523,14.41,68,91,16.7


# Import WOE data

In [4]:
#Open given sheet
worksheet = gc.open('WEO_Data').sheet1

# Read contents of CSV file
WEO_rows = worksheet.get_all_values()

# Convert to a DataFrame and render.
import pandas as pd
weo  = pd.DataFrame.from_records(WEO_rows, columns = WEO_rows[0])

# Drop the header row from data
weo = weo.reindex(weo.index.drop(0))

# 1- Select only Country name and 2015 
# 2- then rename it to GDP Per capita
weo = weo[['Country','2015']].rename(columns={'2015':'GDP per capita'})

# Set Country as index column
# Inplace command, will replace the results of command into the same DF
weo.set_index("Country", inplace=True)

#weo.drop_duplicates(inplace=True)
#Print top 5 rows
weo.head()

,GDP per capita
Country,
Afghanistan,599.99
Albania,3995.38
Algeria,4318.14
Angola,4100.32
Antigua and Barbuda,14414.30


In [5]:
bli["Life satisfaction"].head()

Country
Australia    7.3
Austria        7
Belgium      6.9
Brazil       6.6
Canada       7.3
Name: Life satisfaction, dtype: object

# Merge/Join dataset

In [6]:
df = pd.merge(left = weo, right = bli, left_index=True, right_index=True)
df.sort_values(by="GDP per capita", inplace=True)
df.head()


,GDP per capita,Air pollution,Dwellings without basic facilities,Educational attainment,Employees working very long hours,Employment rate,Feeling safe walking alone at night,Homicide rate,Household net adjusted disposable income,Household net financial wealth,Housing expenditure,Labour market insecurity,Life expectancy,Life satisfaction,Long-term unemployment rate,Personal earnings,Quality of support network,Rooms per person,Self-reported health,Stakeholder engagement for developing regulations,Student skills,Time devoted to leisure and personal care,Voter turnout,Water quality,Years in education
Country,,,,,,,,,,,,,,,,,,,,,,,,,
Luxembourg,101994.09,12,0,79,3.76,66,72,0.6,41317,74141,20,3.2,82.4,6.9,1.9,62636,92,2,70,1.5,483,15.15,91,85,15.1
Hungary,12239.89,19,4.3,83,3.05,67,50.7,1.2,16821,23289,18,4.8,75.7,5.3,2.42,21711,84,1.2,56,1.2,474,15.06,62,76,16.6
Poland,12495.33,22,2.7,91,6.68,65,66.3,0.8,18906,14997,23,4.3,77.6,6,2.14,25921,89,1.1,58,2.6,504,14.42,55,80,17.7
Chile,13340.91,16,9.4,65,10.06,62,51.1,4.5,16588,21409,18,8.1,79.1,6.7,2.02,28434,84,1.9,57,1.5,443,14.9,49,69,17.3
Latvia,13618.57,11,12.9,89,2.09,69,60.7,6.6,15269,17105,23,6.8,74.6,5.9,3.73,22389,86,1.2,46,2.4,487,13.83,59,77,17.9


In [7]:
test_indices = [0, 1, 6, 8, 33, 34, 35]
train_indices = list(set(range(36)) - set(test_indices))

train = df[["Air pollution", 'Life satisfaction']].iloc[train_indices]
test = df[["Air pollution", 'Life satisfaction']].iloc[test_indices]



In [8]:
test


,Air pollution,Life satisfaction
Country,,
Luxembourg,12,6.9
Hungary,19,5.3
Czech Republic,20,6.6
Greece,18,5.2
Switzerland,15,7.5
Brazil,10,6.6
Mexico,16,6.6


In [ ]:
# Code example
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
# Prepare the data
X = np.c_[train["Air pollution"]]
y = np.c_[train["Life satisfaction"]]

# Visualize the data
#df.plot(kind='scatter', x="GDP per capita", y='Life satisfaction')
#plt.show()

# Build a LinearRegression model
model_NormalEquation= ....

# Build a SGDRegressor model (keep maximum iteration at 300 and eta = 0.0001)
model_SGD = ....

# Make a prediction for Cyprus
X_new = [[18]]  # Cyprus' GDP per capita
print(model_NormalEquation.predict(X_new)) # outputs [[5.95199478]]
print(model_SGD.predict(X_new)) # outputs [[5.95199478]]

[[6.25469406]]
[7.40059138]


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
# Make a prediction for our test data
# Predict using your Normal Equation model
pred_NE = ....(test['Air pollution'].values.reshape(-1,1))

# Predit using your SGD model
pred_SGD = ...(test['Air pollution'].values.reshape(-1,1))

In [ ]:
pred_NE.ravel()

array([6.65395503, 6.18815056, 6.12160706, 6.25469406, 6.45432454,
       6.78704202, 6.38778105])

In [ ]:
pred_SGD


array([4.96330678, 7.80680547, 8.21301957, 7.40059138, 6.18194908,
       4.15087858, 6.58816318])

In [ ]:
from sklearn.metrics import mean_squared_error
# Fill in the blank and calculate the error
MAE_NE =mean_squared_error(test['Life satisfaction'] , ....) 
MAE_NE

0.48057823693051915

In [ ]:
# Fill in the blank and calculate the error

MAE_SGD =mean_squared_error(test['Life satisfaction'] , ....) 
MAE_SGD

3.60212613035068

# Discussion & Conclusions:

Compare the results of M
Perhaps this is a very synthetic dataset as the number of test data is 6 only. But, the purpose is that different algorithm works with different performanre and it is up to you to run different model for varied dataset and pick the best!

As you know Normal Equation is mathematically finding global minimum and is always more succseful than SGD (when dataset is snall enough) which is randomly trying to find global minimal.